In [1]:
from transformers import BertModel, BertTokenizer
import torch

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip3 install -q transformers

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [3]:
!pip3 install -q torch

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [4]:
!pip3 install -q pandas
!pip3 install -q numpy
!pip3 install -q scikit-learn

You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Library/Frameworks/Python.framework/Versions/3.10/bin/python3.10 -m pip install --upgrade pip' command.


In [5]:
# Importing stock ml libraries
import numpy as np
import pandas as pd
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertModel, BertConfig
from  tqdm import tqdm
# Preparing for TPU usage
# import torch_xla
# import torch_xla.core.xla_model as xm
# device = xm.xla_device()

In [6]:
# # Setting up the device for GPU usage

from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cpu'

In [7]:
# Sections of config

# Defining some key variables that will be used later on in the training
#MAX_LEN = 200 #200 tokens per sequence?
TRAIN_BATCH_SIZE = 8
VALID_BATCH_SIZE = 4
EPOCHS = 1
LEARNING_RATE = 1e-05
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [8]:
model = BertModel.from_pretrained('bert-base-uncased')


model.safetensors: 100%|██████████| 440M/440M [01:24<00:00, 5.24MB/s] 


In [9]:
input = "Software Engineer, Python, React, San Jose, Computer Science"

In [12]:
# Tokenization and padding
tokens = tokenizer(input, padding=True, truncation=True, return_tensors='pt')

# Model inference
with torch.no_grad():
    outputs = model(**tokens)
print(outputs)
# Extract embeddings (using mean pooling over tokens)
text_embedding = outputs.last_hidden_state.mean(dim=1).squeeze()

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3559, -0.0148, -0.3292,  ...,  0.1947,  0.2486,  0.7107],
         [-0.0134,  0.1188, -0.0250,  ...,  0.0549,  0.0149,  0.3010],
         [ 0.8771, -0.0905, -0.6612,  ..., -0.2598,  0.4075, -0.2933],
         ...,
         [-0.5637,  1.1336,  0.5909,  ..., -0.0183, -0.1920,  0.0214],
         [-0.5398,  0.3534, -0.4607,  ..., -0.0263,  0.4349, -0.0161],
         [ 0.7671, -0.0332, -0.5101,  ...,  0.2722, -0.6717,  0.0052]]]), pooler_output=tensor([[-0.8781, -0.5701, -0.9807,  0.8126,  0.8870, -0.3795,  0.8533,  0.2222,
         -0.8684, -1.0000, -0.8271,  0.9896,  0.9657,  0.6874,  0.7117, -0.8880,
         -0.6384, -0.6443,  0.3732, -0.1860,  0.7411,  1.0000, -0.3539,  0.4617,
          0.5714,  0.9971, -0.8667,  0.7512,  0.9140,  0.7874, -0.5802,  0.5133,
         -0.9890, -0.3215, -0.9681, -0.9912,  0.5567, -0.6949,  0.0258, -0.2557,
         -0.7496,  0.6097,  1.0000,  0.0093,  0.6021, -0.2918, -1.0000,  0.

In [13]:
# Convert embeddings list to a tensor
print(text_embedding.shape)
print(text_embedding)

torch.Size([768])
tensor([-2.5201e-02,  2.9465e-01, -6.0823e-02,  4.1923e-02,  4.6346e-01,
        -1.3923e-01, -6.8516e-02,  7.7270e-01, -2.4056e-01,  1.7613e-01,
         1.5925e-01,  2.8155e-01,  7.9582e-02,  1.5922e-01,  6.4358e-02,
        -1.0745e-01,  1.0831e-01, -2.9678e-03, -2.8732e-02,  5.3749e-01,
        -3.1573e-01,  6.0020e-02,  5.0165e-02,  3.2497e-01,  1.8396e-01,
         2.8662e-01, -5.2252e-01, -3.2555e-01, -8.4684e-01, -1.2933e-01,
        -4.5505e-01,  3.1115e-01, -3.6684e-01, -4.3491e-01,  1.4297e-02,
        -1.5185e-01, -4.3958e-02,  2.7537e-01, -2.6146e-01,  5.1894e-01,
        -6.1521e-01, -6.1271e-01,  2.6688e-01,  2.2958e-01, -3.7415e-02,
        -2.5193e-01,  1.7491e-01,  2.4626e-01,  1.4998e-02, -9.0518e-03,
        -5.9831e-01,  6.8125e-01, -4.8701e-02, -8.1613e-01, -4.4423e-02,
         7.0076e-01,  2.2410e-01, -7.3755e-01, -9.7124e-02, -1.6359e-01,
         2.4109e-02,  5.7907e-01, -1.1623e-01, -1.8109e-01,  3.9167e-01,
         3.3006e-01,  2.5296e-02,